<a href="https://colab.research.google.com/github/gvogiatzis/CS3320/blob/main/CS3320_Lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS3MIR Lab 1. Boolean Retrieval

## Introduction
In this lab we will explore some of the fundamental information retrieval concepts we saw in the lectures including indexing, ranked and boolean retrieval. You are given a small dataset of 737 news stories scraped from the **BBC Sports website** between 2004 and 2005 (full dataset [here](http://mlg.ucd.ie/datasets/bbc.html)). Your task is to write a basic search engine in Python, using the Boolean Retrieval model we talked about in lectures. 
Your engine should consist of an indexer which processes all documents and creates an index as well a query engine that takes a query string and returns a list of documents that contains all the terms.

In [1]:
!wget https://github.com/TharinduDR/CS3MIR/raw/main/data/bbc_sport_docs.zip

--2023-01-25 17:15:45--  https://github.com/TharinduDR/CS3MIR/raw/main/data/bbc_sport_docs.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TharinduDR/CS3MIR/main/data/bbc_sport_docs.zip [following]
--2023-01-25 17:15:45--  https://raw.githubusercontent.com/TharinduDR/CS3MIR/main/data/bbc_sport_docs.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1104592 (1.1M) [application/zip]
Saving to: ‘bbc_sport_docs.zip’

bbc_sport_docs.zip  100%[===================>]   1.05M  --.-KB/s    in 0.02s   

2023-01-25 17:15:45 (56.7 MB/s) - ‘bbc_sport_docs.zip’ saved [1104592/1104592]



In [2]:
!unzip bbc_sport_docs.zip -d docs

Archive:  bbc_sport_docs.zip
replace docs/000.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: docs/000.txt            
  inflating: docs/__MACOSX/._000.txt  
  inflating: docs/001.txt            
  inflating: docs/__MACOSX/._001.txt  
  inflating: docs/002.txt            
  inflating: docs/__MACOSX/._002.txt  
  inflating: docs/003.txt            
  inflating: docs/__MACOSX/._003.txt  
  inflating: docs/004.txt            
  inflating: docs/__MACOSX/._004.txt  
  inflating: docs/005.txt            
  inflating: docs/__MACOSX/._005.txt  
  inflating: docs/006.txt            
  inflating: docs/__MACOSX/._006.txt  
  inflating: docs/007.txt            
  inflating: docs/__MACOSX/._007.txt  
  inflating: docs/008.txt            
  inflating: docs/__MACOSX/._008.txt  
  inflating: docs/009.txt            
  inflating: docs/__MACOSX/._009.txt  
  inflating: docs/010.txt            
  inflating: docs/__MACOSX/._010.txt  
  inflating: docs/011.txt            
  inflating: docs/__MACO

## The dataset
Before we do anything else, let's have a look at the dataset I will assume you will have downloaded from blackboard the dataset *bbcsport.tar.gz* and unzipped it (`gunzip bbcsport.tar.gz` followed by `tar -xf bbcsport.tar`) in a convenient location. Go into that directory and have a look at some of the files using the `cat` command, i.e.

In [3]:
!cat docs/000.txt

McCall earns Tannadice reprieve

Dundee United manager Ian McCall has won a reprieve from the sack, with chairman Eddie Thompson calling for an end to speculation over his future.

It is understood that McCall would have been sacked if Sheffield Wednesday manager Paul Sturrock had been willing to return to Tannadice. But Sturrock has distanced himself from the position. "We're in a difficult situation. We must get out of it through the efforts of current personnel," said Thompson. "Ian McCall and I have had a long and detailed talk about a number of areas including the current league position and the manner of the exit from the League Cup," he added. "However, the continuing speculation is doing no one any good, especially as we have several crucial games coming up. "The minds of the coaching staff and the players have to be on those games and those games only. "Our season would of course improve considerably if in the next few weeks we achieved some improved league results and there i

should produce something similar to

    McCall earns Tannadice reprieve

    Dundee United manager Ian McCall has won a reprieve from the sack, with chairman Eddie Thompson calling for an end to speculation over his future...etc

Now let's try to open and print the file in python. First, you need to import these modules that will be used throughout

In [4]:
import re
from glob import glob

We can open a file using the built-in `open` command as follows:

In [5]:
f = open('docs/000.txt','r', encoding='latin-1')

The method f.read() then reads the file into a string as follows

In [6]:
s = f.read()

Try printing the first 200 characters to see how the output looks. 

In [7]:
print(s[0:199])

McCall earns Tannadice reprieve

Dundee United manager Ian McCall has won a reprieve from the sack, with chairman Eddie Thompson calling for an end to speculation over his future.

It is understood t


Try to open and print some part of another file in the box below

We can use the `glob()` command to obtain a list of all the filenames under a directory and then open a specific one using an index into that list. This index will become our document ID that our search engine will use. All that can be neatly placed inside a function for printing files as follows:

In [8]:
filenames = sorted(glob("docs/*.txt"))

def readfile(fname):
    f = open(fname, 'r', encoding='latin-1')
    s = f.read()
    f.close()
    return s

print(readfile(filenames[0]))

McCall earns Tannadice reprieve

Dundee United manager Ian McCall has won a reprieve from the sack, with chairman Eddie Thompson calling for an end to speculation over his future.

It is understood that McCall would have been sacked if Sheffield Wednesday manager Paul Sturrock had been willing to return to Tannadice. But Sturrock has distanced himself from the position. "We're in a difficult situation. We must get out of it through the efforts of current personnel," said Thompson. "Ian McCall and I have had a long and detailed talk about a number of areas including the current league position and the manner of the exit from the League Cup," he added. "However, the continuing speculation is doing no one any good, especially as we have several crucial games coming up. "The minds of the coaching staff and the players have to be on those games and those games only. "Our season would of course improve considerably if in the next few weeks we achieved some improved league results and there i

Try using this function on the `docs` path for various files in the box below: 

So we now know how to open a file and turn its contents into a string. The next step is chopping that string up in tokens.


## Tokenisation
Fortunately tokenization is a very simple task in Python because we can use the built-in `split` method for strings. Let's try reading a file into a string, split it and then print the first 20 tokens.

In [9]:
s = readfile(filenames[0])
tokens = s.split()
print(tokens[0:19])

['McCall', 'earns', 'Tannadice', 'reprieve', 'Dundee', 'United', 'manager', 'Ian', 'McCall', 'has', 'won', 'a', 'reprieve', 'from', 'the', 'sack,', 'with', 'chairman', 'Eddie']


We see that the simple `split` method in strings only splits on whitespace characters leaving punctuation marks (as well as numbers, hyphens etc) which is why it returned "`sack,`" as a token. 

Let's use an advanced tokeniser to perform tokenisation. We can do this easily with NLTK. NLTK (Natural Language Toolkit) is the go-to API for NLP (Natural Language Processing) with Python. It is a powerful tool to preprocess text data for further analysis for ML (Machine Learning) models. 

NLTK is already installed is colab environment. Let's check the version.

In [10]:
import nltk

print('The NLTK version is {}.'.format(nltk.__version__))

The NLTK version is 3.7.


For the tokenisation we additionally need the following resource, which is needed to be downloaded through NLTK.

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Let's perform tokenisation with NLTK

In [12]:
from nltk.tokenize import word_tokenize

tokens = word_tokenize(s)
print(tokens[0:19])


['McCall', 'earns', 'Tannadice', 'reprieve', 'Dundee', 'United', 'manager', 'Ian', 'McCall', 'has', 'won', 'a', 'reprieve', 'from', 'the', 'sack', ',', 'with', 'chairman']


You can see that NLTK is better in tokenisation than the split function as it returned "sack" and "," as two tokens. 

## Stemming

We can use NLTK to perform stemming. We will use the Porter Stemmer that we saw in the lecture.

In [13]:
from nltk.stem import PorterStemmer


ps = PorterStemmer()

stemmed_tokens = []
for token in tokens:
    rootWord = ps.stem(token)
    stemmed_tokens.append(rootWord)

print(stemmed_tokens[0:19])

['mccall', 'earn', 'tannadic', 'repriev', 'dunde', 'unit', 'manag', 'ian', 'mccall', 'ha', 'won', 'a', 'repriev', 'from', 'the', 'sack', ',', 'with', 'chairman']


Spot the differences Porter Stemmer made to the tokens. 

There is another stemmer in NLTK that is called Snowball Stemmer. Snowball Stemmer is also known as the Porter2 stemming algorithm because it is a better version of the Porter Stemmer. It is more aggressive than Porter Stemmer. 

In the following cell use the documentation here - https://www.nltk.org/api/nltk.stem.snowball.html and stem the same text above using Snowball Stemmer.

## Lemmatisation

We can use NLTK to perform lemmatisation too. We will use the popular WordNet Lemmatiser. We need to download wordnet and omw through NLTK to do that.

In [14]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
import nltk
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

lemmatised_tokens = []
for token in tokens:
  lemmatised_token = wordnet_lemmatizer.lemmatize(token)
  lemmatised_tokens.append(lemmatised_token)

print(lemmatised_tokens[0:19])
		

['McCall', 'earns', 'Tannadice', 'reprieve', 'Dundee', 'United', 'manager', 'Ian', 'McCall', 'ha', 'won', 'a', 'reprieve', 'from', 'the', 'sack', ',', 'with', 'chairman']


## Boolean retrieval - Indexing
In this section we will look at Boolean retrieval, as a warming up excercise before you tackle ranked retrieval on your own! The first step in a boolean search engine is to build the indexer. This piece of code is responsible for reading all the documents and producing the postings lists which, as we saw in the lectures, is an efficient way of storing the term-document incidence matrix.

A nice data-structure for storing postings lists is a Python dictionary which as we have seen can be indexed by strings. So we want to produce a dictionary whose keys are words found in the documents and whose values are *sets* of docid's of documents that contain those words. So a postings list such as
    
    {'cricket': {2, 3, 5, 7}, 'football': {0, 2, 4}, 'rugby': {1, 2, 6}}
    
would denote that the word *cricket* can be found in documents with id's 2, 3, 5 and 7 etc. So how to construct this postings dictionary? Let us read and tokenize the file with id 0 once again. For the codes below we are only using tokenising. As homework redo the code with lemmatisation and stemming too. 

In [16]:
s = readfile(filenames[0])
words = word_tokenize(s)

Now starting with an empty dictionary we will add all the words in `words`. If the word is not contained in the dictionary we create a singleton set with the doc-id 0. If the word is already contained in the dictionary we only add 0 to the corresponding set of docid's. This looks as follows: 

In [17]:
postings = {}
for w in words:
    postings.setdefault(w, set()).add(0)

Print out the `postings` dictionary in the box below to see what it contains. You should get a dictionary with keys equal to all the words contained in our doc and values {0}. 



We can now do this for the whole collection of documents, and encapsulate the whole indexing engine in a function as follows:

In [18]:
def indextextfiles_BR(path):
    filenames = sorted(glob(path))
    postings={}         
    for docID,fname in enumerate(filenames):
        s = readfile(fname)
        words = word_tokenize(s)
        for w in words:
            if w!='':
                postings.setdefault(w, set()).add(docID)
    return postings

So to process the entire directory and generate the complete postings dictionary we can execute:

In [19]:
postings = indextextfiles_BR('docs/*.txt')

Let's now use this datastructure to find out which documents contain the word '`devastating`'. We just need to execute:

In [20]:
print(postings['devastating'])

{195, 310, 55}


which tells us that the word 'devastating' is contained in docs with id's 195, 310 and 55. Neat! Use the box below to print the postings list of other words. Can you find a word that is contained in only one document?

## Processing boolean queries
We are now ready to process a boolean query. We will be assuming that the user gives a string containing all the query terms separated by spaces and they will expect all documents that contain all those query terms. Let's begin by assuming the query text '`england football defeat`'

In [21]:
qtext = 'England football defeat'

Let's begin by tokenizing it

In [22]:
words = word_tokenize(qtext)
print(words)

['England', 'football', 'defeat']


To perform boolean retrieval we need to get the postings of each of the three terms and then take their intersection (`&` operator in Python) as follows:

In [23]:
p1 = postings['England']
p2 = postings['football']
p3 = postings['defeat']
print(p1&p2&p3)

{682, 602, 597, 610}


More generally, we can iterate through the list of terms in the query, grab the postings sets for each of them and take their intersection. This can be done inside a function as follows. 

In [24]:
def query_BR(postings, qtext):
    words = word_tokenize(qtext)
    res = None
    for w in words:
        if res == None:
            res = postings[w]
        else:    
            res = res & postings[w]
    return res

Notice the check inside the for-loop for the very first time through the loop where `res` has not been set yet. If you want, you can try a more elegant (or Pythonic as people call it) way of doing the same thing  using `set.intersection` and the `*` operator that turns a list into arguments for a function. 

In [25]:
def query_BR(postings, qtext):
    words = word_tokenize(qtext)
    allpostings = [postings[w] for w in words]
    res = set.intersection(*allpostings)
    return res

Let's see if we get the same results

In [26]:
query_BR(postings,'England football defeat')

{597, 602, 610, 682}

Nice!

In the box below, run a different query and then verify that the results are correct by printing out the text and checking if the terms are contained in all documents.

Please save this as pdf by going to File->Print and select destination as "Save to pdf". You can then upload the file to blackboard using the submission link found next to the lab spec.